# AgentNeo: Getting Started

In [ ]:
!pip install agentneo -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 952.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.5/288.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succe

In [ ]:
from agentneo import AgentNeo, Project, Dataset, Experiment, Tracer

In [ ]:
# To login using email-id, creates new account if is not a user else retrieves the existing account
BASE_URL = "http://74.249.60.46:5000"

agent_session = AgentNeo(email="user1@example.com", base_url=BASE_URL)

Existing user found for email: user1@example.com
Please keep your access key and secret key handy for future logins:
Access Key: 2USyfi19AxtWqtuo3WTGww
Secret Key: F_YZRdqZcPk5Vz7YXT4I4VGRin0e9C4T_sqOBDHiOIc


In [ ]:
access_key = "2USyfi19AxtWqtuo3WTGww"
secret_key = "F_YZRdqZcPk5Vz7YXT4I4VGRin0e9C4T_sqOBDHiOIc"

# To login with the the access_key, secret_key
agent_session = AgentNeo(
        access_key=access_key,
        secret_key=secret_key,
        base_url=BASE_URL,
    )

## Project

In [ ]:
project_created = Project(session=agent_session, project_name="Project_2", description="A test project").create()
project_id = project_created['id']

An error occurred while creating the project: Request failed: {'error': 'You already have a project with this name'}


KeyError: 'id'

## Tracing

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install langchain_openai langchain_community langgraph arxiv -q

In [ ]:
# Initialise the tracer by providing the tools-used and their description
tracer = Tracer(session=agent_session, metadata={'tools': [
    {'name': 'arxiv_tool', 'description': "Tool to search Arxiv for available research papers."},
    {'name': 'agent', 'description': "Identify the search terms for arxiv from the user description and call the arxiv_tool based on the search term"},
]})

# Langgraph code
from langchain_openai import ChatOpenAI
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from typing import TypedDict, Annotated, Sequence
import operator
from langgraph.graph import StateGraph, END
import os

openai_llm = ChatOpenAI(temperature=0.4, callbacks=[tracer.get_callback_handler()])
arxiv = ArxivAPIWrapper(
    top_k_results=3,
    ARXIV_MAX_QUERY_LENGTH=300,
    load_max_docs=3,
    load_all_available_meta=False,
    doc_content_chars_max=40000
)

class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]

@tracer.trace_node
def agent(state):
    query = state["messages"]
    res = openai_llm.invoke(f"""
    You are given text summary on the research topics that the user is working on.
    You need to extract the search term from the text summary.
    Here is the Text summary: {query[0]}
    """)
    return {"messages": [res.content]}

@tracer.trace_node
def arxiv_tool(state):
    context = state["messages"]
    search_query = context[1]
    data = arxiv.run(search_query)
    return {"messages": [data]}

@tracer.trace_node
def responder(state):
    agent = openai_llm.invoke(f"""
    You are given search results on the research topics that the user is working on.

    Here is the user query:
    ---
    {state["messages"][0]}
    ---

    Here is the search results:
    ---
    {state["messages"][2]}
    ---
    """)
    return {"messages": [agent.content]}
@tracer.trace_node
def where_to_go(state):
    ctx = state["messages"][0]
    if ctx == "no_response":
        return "end"
    else:
        return "continue"

@tracer.trace_graph
def workflow():
    graph = StateGraph(AgentState)
    graph.add_node("agent", agent)
    graph.add_node("arxiv", arxiv_tool)
    graph.add_node("responder", responder)
    graph.add_conditional_edges("agent", where_to_go, {
        "end": END,
        "continue": "arxiv"
    })
    graph.add_edge("agent", "arxiv")
    graph.add_edge("arxiv", "responder")
    graph.set_entry_point("agent")
    graph.set_finish_point("responder")
    return graph.compile()

compiled_workflow = workflow()

@tracer.trace_node
def get_output(message):
    inputs = {"messages": [message]}
    response = compiled_workflow.invoke(inputs)
    print(response["messages"][-1])

get_output("Is someone working on the propagation of LASER through brownian fluids")

Based on the search results provided, it seems that there are researchers working on the propagation of LASER through brownian fluids. 

1. The first study published in 2017 focuses on quantifying the temporal structural diversity of a laser beam propagating through isotropic optical turbulence. The researchers conducted a laboratory-controlled experiment to analyze coordinate fluctuations of the laser beam and observed a transition from an integrated stochastic process to a fractional Brownian motion as the sampling time increased.

2. The second study published in 2021 experimentally studied the impact of inertial reference frame changes on overdamped Brownian motion using a laser to induce convection flows in a column of fluid. The researchers verified the weak Galilean invariance of Brownian dynamics and derived a Galilean invariant expression of the stochastic thermodynamic first law.

3. The third study published in 2008 measured the overall motion of Brownian particles suspended

In [ ]:
trace_id = tracer.upload_trace()

Trace uploaded successfully. Trace ID: 11


## Dataset

In [ ]:
# Create a dataset from a trace
dataset = Dataset(
    session=agent_session,
    project_id=project_id,
    dataset_name="Dataset_1",
    description="A test dataset"
)

dataset_traced = dataset.from_trace(trace_id=tracer.id, trace_filter=None)

## Experiment

In [ ]:
# Create Experiment
experiment = Experiment(
        session=agent_session,
        experiment_name="Experiment_1",
        description="A test experiment",
        dataset_id=dataset_traced['id'],
        project_id=project_id
    )

experiment_created = experiment.create()

In [ ]:
# Execute Experiment
exp = experiment.execute(metrics=[
    {"name": "summarise", "config": {}},
    {"name": "tool_selection_accuracy", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ.get("OPENAI_API_KEY")}},
    {"name": "tool_usage_efficiency", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ.get("OPENAI_API_KEY")}},
    {"name": "goal_decomposition_efficiency", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ.get("OPENAI_API_KEY")}},
    {"name": "plan_adaptibility", "config": {"model": "gpt-4o-mini", "OPENAI_API_KEY": os.environ.get("OPENAI_API_KEY")}},
])

In [ ]:
# Experiment Results
exp = experiment.get_results(experiment_id=exp['id'])

for i in exp['results']:
    print(f"Name: {i['metric_name']}")
    print(f"Result:")
    for key, value in i['result'].items():
        print(f"{key}: {value}")
    print(f"{'*'*100}\n")

Name: summarise
Result:
completion_cost: 0.0032025
completion_tokens: 2135
input_cost: 0.0021349999999999997
input_tokens: 4270
latency: 0.006265878677368164
summary: {'children': 'list with 1 items', 'end': 'NoneType', 'inputs': 'NoneType', 'name': 'str', 'outputs': 'NoneType', 'start': 'NoneType', 'type': 'str'}
total_cost: 0.0053375
total_tokens: 6405
****************************************************************************************************

Name: tool_selection_accuracy
Result:
reasoning: ["The selected tool 'agent' is relevant as it identifies search terms for the arxiv_tool based on the user's inquiry about ongoing research. However, it could be argued that directly using the arxiv_tool would have been more efficient since the user is specifically looking for research papers.", "The selection of the arxiv_tool is optimal as it directly addresses the user's request for research related to the propagation of LASER through Brownian fluids. The tool successfully retrieves r